# Análisis de Negocio

**Objetivo del Modelo:**
El modelo está diseñado para generar texto similar al de "Don Quijote de la Mancha". Sus aplicaciones potenciales incluyen:
- Generación de contenido: Crear textos literarios o inspirados en obras clásicas.
- Asistencia a escritores: Proporcionar ideas o continuaciones de frases.
- Educación: Enseñar sobre generación de lenguaje natural y procesamiento de texto.

**Ventajas:**
- Flexibilidad: Puede adaptarse a otros textos con cambios mínimos.
- Automatización: Genera contenido rápidamente sin intervención humana.
- Personalización: Permite ajustar parámetros como la temperatura para controlar la creatividad del texto generado.

**Limitaciones:**
- Calidad del texto: Aunque el texto generado es coherente, puede carecer de sentido profundo o contexto preciso.
- Dependencia de datos: La calidad del modelo depende en gran medida del texto de entrenamiento.
- Recursos computacionales: El entrenamiento de modelos LSTM es costoso en términos de tiempo y hardware.

**Oportunidades:**
- Integración con herramientas de escritura: Podría incorporarse en editores de texto como sugeridor automático.
- Multilingüismo: Entrenar el modelo con textos en diferentes idiomas para ampliar su alcance.
- Mejoras con modelos avanzados: Migrar a arquitecturas como Transformers para mejorar la calidad del texto generado.

**Riesgos:**
- Sobreajuste: El modelo podría memorizar fragmentos del texto de entrenamiento en lugar de generalizar.
- Uso ético: La generación automática de contenido debe manejarse con cuidado para evitar plagio o desinformación.

# Cargar y preprocesar el texto

In [ ]:
import tensorflow as tf
import numpy as np
import os
import time

path = tf.keras.utils.get_file('quijote.txt', 'https://www.gutenberg.org/files/2000/2000-0.txt')
text = open(path, 'rb').read().decode(encoding='utf-8').lower()

vocab = sorted(set(text))
print(f'{len(vocab)} caracteres únicos')

2226045/2226045 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
74 caracteres únicos


# Mapear caracteres a enteros

Se crean dos diccionarios: **char2idx** para convertir caracteres a índices numéricos y **idx2char** para realizar el proceso inverso. El texto se transforma en una secuencia numérica **(text_as_int)**, lo que permite al modelo procesarlo como una serie de números en lugar de caracteres. Esta representación numérica es esencial para las operaciones matemáticas en el modelo.

In [ ]:
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

# Crear secuencias de entrenamiento

El texto se divide en secuencias de longitud fija **(seq_length = 100)**, donde cada secuencia se usa para predecir el siguiente carácter. La función **split_input_target** separa cada secuencia en una entrada (todos los caracteres excepto el último) y un objetivo (todos los caracteres excepto el primero). Esto permite entrenar al modelo para predecir el siguiente carácter en una secuencia.

In [ ]:
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

# Preparar batches

Las secuencias se agrupan en lotes **(BATCH_SIZE = 64)** y se barajan para mejorar el entrenamiento. El uso de batches permite procesar múltiples secuencias simultáneamente, optimizando el uso de recursos computacionales.

In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Crear el modelo LSTM

In [ ]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    return tf.keras.Sequential([
        tf.keras.layers.Input(batch_shape=(batch_size, None)),
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(vocab_size)
    ])


model = build_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)

El modelo consta de:
- Una capa de Embedding para convertir índices numéricos en vectores densos.
- Una capa LSTM con 1024 unidades, que captura dependencias a largo plazo en las secuencias.
- Una capa de Dropout para regularización, reduciendo el sobreajuste.
- Una capa Dense final para predecir el siguiente carácter.

El diseño del modelo es adecuado para la generación de texto, ya que las LSTMs son efectivas para manejar secuencias y patrones temporales.

# Compilar y entrenar

El modelo se compila con el optimizador Adam y la función de pérdida **sparse_categorical_crossentropy**. Se divide el dataset en entrenamiento (80%) y validación (20%). Se utilizan callbacks como ModelCheckpoint para guardar pesos y **EarlyStopping** para detener el entrenamiento si no hay mejora en la precisión de validación. El entrenamiento muestra una mejora constante en la precisión y una reducción en la pérdida, tanto en entrenamiento como en validación.

In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(
    optimizer='adam',
    loss=loss,
    metrics=[
        tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy")
    ]
)

total_batches = dataset.cardinality().numpy()
train_size = int(0.8 * total_batches)

train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

earlystop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=2,
    restore_best_weights=True
)

EPOCHS = 20
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    callbacks=[checkpoint_callback, earlystop_callback]
)

Epoch 1/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 2213s 8s/step - accuracy: 0.2891 - loss: 2.5213 - val_accuracy: 0.4746 - val_loss: 1.7088
Epoch 2/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 2245s 8s/step - accuracy: 0.4942 - loss: 1.6346 - val_accuracy: 0.5521 - val_loss: 1.4395
Epoch 3/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 2258s 8s/step - accuracy: 0.5511 - loss: 1.4364 - val_accuracy: 0.5827 - val_loss: 1.3315
Epoch 4/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 2236s 8s/step - accuracy: 0.5757 - loss: 1.3480 - val_accuracy: 0.5981 - val_loss: 1.2725
Epoch 5/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 2228s 8s/step - accuracy: 0.5909 - loss: 1.2945 - val_accuracy: 0.6113 - val_loss: 1.2262
Epoch 6/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 2229s 8s/step - accuracy: 0.6009 - loss: 1.2584 - val_accuracy: 0.6214 - val_loss: 1.1918
Epoch 7/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 2238s 8s/step - accuracy: 0.6099 - loss: 1.2284 - val_accuracy: 0.6290 - val_loss: 1.1643
Epoch 8/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 2248s 8s/step - accuracy: 0.6183 - loss: 1.1996 - 

# Funciones para generar texto con el modelo entrenado

Se implementarán funciones para generar texto:
- generate_word: Genera una palabra a partir de un carácter inicial.
- generate_phrase: Genera una frase a partir de una palabra inicial.

El modelo muestra capacidad para generar texto coherente, aunque con algunos errores gramaticales y de contexto.

In [ ]:
def generate_word(model, start_char, temperature=1.0, max_chars=30):
    input_eval = [char2idx[c] for c in start_char.lower() if c in char2idx]
    input_eval = tf.expand_dims(input_eval, 0)
    model.layers[1].reset_states()

    word = start_char
    for _ in range(max_chars):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0) / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        next_char = idx2char[predicted_id]
        word += next_char

        if next_char == ' ':
            break
        input_eval = tf.expand_dims([predicted_id], 0)

    return word.strip()


def generate_phrase(model, start_word, temperature=1.0, max_words=10):
    current_input = start_word
    phrase = start_word.strip()

    for _ in range(max_words - 1):
        next_word = generate_word(model, current_input[-1], temperature=temperature)
        phrase += ' ' + next_word
        current_input = next_word

    return phrase


En un lugar de la mancha, y
que el que de bronce, hecha un pas de los caballones gánedas y
señoras. tú mis lados de la tierra, y mis costillas puntualmente —ue le
hallamos— yo soy el borroso historiador está por la camisa, atóndiente que tiene las niñas o
amadís. ¡bon, tr sinas oblinados niñe-''''.fito que la de cosas, y que mi servente le hubiese
ala y muerto, sin posibilitarse,
al persando y se entraron, don quijote y no habéis de estar
todo, y en la de tantos
fuertes hingas.

— deteneos, soy de mi tridad e


# Evaluación del Modelo

**Resultados del Entrenamiento:**

**Precisión (Accuracy):**
- Entrenamiento: Aumentó de ~29% en la primera época a ~68% en la última.
- Validación: Mejoró de ~47% a ~69%, mostrando que el modelo generaliza bien.

**Pérdida (Loss):**
- Entrenamiento: Reducción de 2.52 a 1.01.
- Validación: Reducción de 1.71 a 0.97.

**Interpretación:**
- El modelo aprendió efectivamente a predecir el siguiente carácter en una secuencia, como lo demuestra el aumento en la precisión y la reducción en la pérdida.
- La pequeña brecha entre las métricas de entrenamiento y validación indica que no hubo sobreajuste significativo.

**Generación de Texto:**

- Fortalezas: El texto mantiene un estilo similar al original, con estructura gramatical aceptable.
- Debilidades: Algunas palabras o frases carecen de sentido (ej. "caballones gánedas").

**Cómo mejorariamos el modelo:**
- Ajustar Hiperparámetros: Probar con más unidades LSTM o mayor dimensión de embedding para mejorar la calidad del texto.
- Aumentar Datos: Incluir más textos literarios para enriquecer el vocabulario y contexto.
- Regularización: Aumentar el dropout o usar técnicas como weight decay para evitar sobreajuste.

**Conclusión:**
El modelo logra su objetivo de generar texto inspirado en "Don Quijote", con un rendimiento sólido en términos de precisión y pérdida. Sin embargo, hay margen para mejorar la coherencia y relevancia del texto generado. Con ajustes y más datos, podría convertirse en una herramienta poderosa para aplicaciones creativas y educativas.